In [304]:
import pandas as pd
import numpy as np
from dateutil.parser import parse as dateparser
from numpy import isnan
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

In [217]:
# if earlier than 1999 year (there is 1900 and 1899) return -1, else return number 
# of days since 1999-1-1
def transform_date(date):
    if pd.isnull(date):
        return date
    else:
        try:
            pivot = dateparser('1999-1-1', dayfirst=False)
            current = dateparser(date, dayfirst=False)
            if current.year < 1999:
                return -1
            else:
                delta = current - pivot
                return delta.days
        except Exception as inst:
            print 'Exception: ' + str(date)
            print type(inst)
            print inst.args

In [218]:
path = '/Users/tgaponov/Desktop/cisco/challenge/'
path = path + 'data/small_training_set.tsv'

data = pd.read_csv(path, delimiter='\t')

features_list = []

In [219]:
# a column should have 60 % and more not null entries
thresh = int(len(data)*0.6)

In [220]:
data = data.loc[:, (data != data.iloc[0]).any()] 
data = data.dropna(axis=1, how='all')
data = data.dropna(axis=1, thresh=thresh)
len(list(data))

182

In [221]:
dates_columns = [x for x in list(data) if 'date' in x]
data[dates_columns] = data[dates_columns].applymap(transform_date)
features_list += dates_columns
# getting all dates columns
#dates = list(data[dates_columns].stack().reset_index()[0])
#dates = [dateparser(x) for x in dates]

In [222]:
# yes - no columns
yorn_columns = [x for x in list(data) if 'yorn' in x]
yorn_encoder = LabelEncoder()
data[yorn_columns] = data[yorn_columns].apply(yorn_encoder.fit_transform)
features_list += yorn_columns

In [231]:
# name columns
name_columns = [x for x in list(data) if 'name' in x]
name_dict = defaultdict(LabelEncoder)
data[name_columns] = data[name_columns].apply(lambda x: name_dict[x.name].fit_transform(x))
features_list += name_columns

In [249]:
# numerical columns
num_columns = ['PRODUCT_TRANSACTION_TYPE',
'CONTRACT_LINE_DURATION_IN_DAYS',
'SERVICE_CONTRACT_DISCOUNT_PERCENTAGE',
'CONTRACT_LINE_REACTION_TIME_CODE',
'SALES_HIERARCHY_LEVEL',
'SERVICE_SALES_NODE_BASE_SALES_HIERARCHY_LEVEL',
'SERVICE_FEE_AMOUNT',
'MAPPED_SERVICE_LIST_PRICE',
'SERVICE_PRODUCT_BASE_SERVICE_FEE_AMOUNT',
'SERVICE_PRODUCT_BASE_MAPPED_SERVICE_LIST_PRICE',
'CONTRACT_LINE_NET_USD_AMOUNT',
'PRODUCT_NET_PRICE',
'SERVICE_PARTNER_INSTALLED_BASE_PARTNER_RENEWAL_RATE',
'SERVICE_SALES_NODE_INSTALLED_BASE_SALES_NODE_RENEWAL_RATE',
'PRODUCT_RENEWAL_RATE',
'PARTNER_RENEWAL_RATE',
'CUSTOMER_RENEWAL_RATE',
'SALES_NODE_RENEWAL_RATE']

In [263]:
num_columns = ['ibsa_main_small_training_set.' + x.lower() for x in num_columns]
num_columns = [x for x in num_columns if x in list(data)]
features_list += num_columns

In [269]:
# type columns
type_columns = [x for x in list(data) if 'type' in x]
type_dict = defaultdict(LabelEncoder)
data[type_columns] = data[type_columns].apply(lambda x: type_dict[x.name].fit_transform(x))
features_list += type_columns

In [271]:
# type columns
code_columns = [x for x in list(data) if 'code' in x]
code_dict = defaultdict(LabelEncoder)
data[code_columns] = data[code_columns].apply(lambda x: code_dict[x.name].fit_transform(x))
features_list += code_columns

In [279]:
remaining_features = [x for x in list(data) if x not in features_list]

In [296]:
final_dataset = data[features_list]


In [322]:
clf = RandomForestClassifier(random_state=29)

X = final_dataset.loc[:, final_dataset.columns != 'ibsa_main_small_training_set.renewed_yorn']
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X_ = imp.fit_transform(X)

y = final_dataset['ibsa_main_small_training_set.renewed_yorn']

X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.30, random_state=129)

clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=29, verbose=0, warm_start=False)

In [323]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.97981333333333331

In [319]:
importance = zip(clf.feature_importances_, list(X))

In [325]:
path = '/Users/tgaponov/Desktop/cisco/challenge/'
path = path + 'data/scoring_set.tsv'

scoring = pd.read_csv(path, delimiter='\t', engine='python')

ParserError: Expected 237 fields in line 94517, saw 274